In [37]:
import pandas as pd
import logging
import numpy as np
from icecream import ic
from itertools import combinations
from geopy.distance import geodesic
from dataclasses import dataclass
import functools
import random
from tqdm import tqdm

logging.basicConfig(level=logging.DEBUG)

In [38]:
PATH = "./cities/"
INSTANCE = "russia.csv"

In [39]:
CITIES = pd.read_csv(f"{PATH}{INSTANCE}", header=None, names=['name', 'lat', 'lon'])

DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

## Helper functions

In [ ]:
def counter(fn):
    """Simple decorator for counting number of calls"""
    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)
    helper.calls = 0
    return helper

@counter
def tsp_cost(tsp):
    #ic(tsp[0], tsp[-1])
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

def fitness(individual):
    return  -float(tsp_cost(individual.genome))

def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]
        
@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None

In [41]:
def greedy_tsp(city):
    visited = np.full(len(CITIES), False)
    dist = DIST_MATRIX.copy()
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        #logging.debug(f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)")
        visited[closest] = True
        city = closest
        tsp.append(int(city))
        #logging.debug(f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)")
    tsp.append(tsp[0])
   # logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")
    return tsp

In [ ]:
# Partially Mapped Crossover
def pmover(p1, p2):
    i1 = np.random.randint(len(p1))
    i2 = np.random.randint(len(p1))
    while i1 == i2:
        i2 = np.random.randint(len(p1))
    #ic(i1, i2)
    if i1 > i2:
        i1, i2 = i2, i1
 
    o1 = p1.copy()
    o2 = p2.copy()
    o1[i1:i2] = p2[i1:i2]
    o2[i1:i2] = p1[i1:i2]

    for i in range(0, i1):
        while o1[i] in o1[i1:i2]:
            o1[i] = p1[p2.index(o1[i])]
        while o2[i] in o2[i1:i2]:
            o2[i] = p2[p1.index(o2[i])]

    for i in range(i2, len(o1)):
        while o1[i] in o1[i1:i2]:
            o1[i] = p1[p2.index(o1[i])]
        while o2[i] in o2[i1:i2]:
            o2[i] = p2[p1.index(o2[i])]
    
   # ic(p1,p2)
   # ic(o1,o2)
    return o1, o2


In [ ]:
# Inversion Mutation
def inv_mutation(individual):
    mutated_genome = individual.genome[:-1]
    i, j = sorted(np.random.randint(0, len(individual.genome), 2))
    while i == j:
        i, j = sorted(np.random.randint(0, len(individual.genome), 2))
   # ic(i, j)
    mutated_genome[i:j+1] = mutated_genome[i:j+1][::-1]
    return mutated_genome
    
def generate_random_individuals(initial_individual):
    new_genome = random.sample(initial_individual.genome[:-1], len(initial_individual.genome[:-1]))
    new_genome.append(new_genome[0])
    return Individual(new_genome)

## Inversion mutation with partially mapped crossover

In [ ]:
POPULATION_SIZE = int(len(CITIES)*1.5)
OFFSPRING_SIZE = int(POPULATION_SIZE*0.25)
MAX_GENERATIONS = 10_000
population = list()

# generate initial individual using a greedy starting from a random city
initial_individual = Individual(greedy_tsp(np.random.randint(len(CITIES))))

# generate random initial population starting from the initial individual
population = [generate_random_individuals(initial_individual) for _ in range(POPULATION_SIZE-1)]
# add the initial individual
population.append(initial_individual)

population.sort(key=lambda i: tsp_cost(i.genome))
# Display cost of best initial individual
ic(tsp_cost(population[0].genome))

for i in population:
    i.fitness = -tsp_cost(i.genome)

for g in tqdm(range(MAX_GENERATIONS)):
    offspring = list()
    for _ in range(OFFSPRING_SIZE):
        #Selection of parents
        p1 = parent_selection(population)
        p2 = parent_selection(population)
        
        #Mutation of parents
        inv_p1 = inv_mutation(p1)
        inv_p1.append(inv_p1[0])
        inv_p2 = inv_mutation(p2)
        inv_p2.append(inv_p2[0])
        
        #Crossover of mutated parents
        o1,o2 = pmover(inv_p1[:-1], inv_p2[:-1])

        o1.append(o1[0])
        o2.append(o2[0])

        o1,o2 = Individual(o1), Individual(o2)

        offspring.append(o1)
        offspring.append(o2)

    for i in offspring:
        i.fitness = fitness(i)
        # ic(i.genome)
        # ic(i.fitness)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]



for i in population:
    # Add last one
    i.genome = np.append(i.genome, i.genome[0])
    i.fitness = -tsp_cost(i.genome)

population.sort(key=lambda i: i.fitness, reverse=True)
# Display cost of best individual after evolution
ic(-population[0].fitness, tsp_cost.calls)

ic| tsp_cost(initial_individual.genome): np.float64(43716.77908582314)
ic| tsp_cost(population[0].genome): np.float64(43716.77908582314)
100%|██████████| 10000/10000 [23:17<00:00,  7.16it/s] 
ic| -population[0].fitness: np.float64(34408.65365296137)
    tsp_cost.calls: 1240752


(np.float64(34408.65365296137), 1240752)